## Define mapper map()

In [82]:
%%file mapper.py
import io
import sys

input_stream = io.TextIOWrapper(sys.stdin.buffer, encoding='latin1')

# Your code setting up data structures here if necessary (equivalent to setup() function)

for line in input_stream:
    # Your code operating on each line here (equivalent to map() function)
    collectionID, doc = line.strip().split()
    print(f"{doc}\t{collectionID}")

# Your code for post-processing here if necessary (equivalent to cleanup() function)

Overwriting mapper.py


## Define reducer reduce()

In [83]:
%%file reducer.py
import sys

# Your code setting up data structures here if necessary (equivalent to setup() function)
table = {}

for line in sys.stdin:
    # Your code operating on each line here (more or less equivalent to reduce() function,
    # but only operating on a single tuple at a time), pairs are sorted by key
    print(f'{line}', file=sys.stderr)
    doc, collectionID = line.strip().split('\t')
    collectionID = int(collectionID)
    if doc not in table:
        table[doc] = [collectionID]
    else:
        table[doc] += [collectionID]

for doc, collectionIDs in table.items():
    if 0 in collectionIDs and 1 in collectionIDs:
        print(f"{doc}\t1")

Overwriting reducer.py


## Define combiner (part of mapper) combine()

In [84]:
%%file combiner.py
import sys

# Your code setting up data structures here if necessary (equivalent to setup() function)
table = {}

for line in sys.stdin:
    # Your code operating on each line here (more or less equivalent to reduce() function,
    # but only operating on a single tuple at a time), pairs are sorted by key
    print(f'{line}', file=sys.stderr)
    doc, collectionID = line.strip().split('\t')
    collectionID = int(collectionID)
    if doc not in table:
        table[doc] = [collectionID]
    else:
        table[doc] += [collectionID]

for doc, collectionIDs in table.items():
    if 0 in collectionIDs:
        print(f"{doc}\t0")
    if 1 in collectionIDs:
        print(f"{doc}\t1")

Overwriting combiner.py


In [85]:
!chmod u+rwx mapper.py
!chmod u+rwx reducer.py
!chmod u+rwx combiner.py

In [91]:
%%file doc.txt
0 hellohello
0 hadoop
0 hadoop
1 hello
1 hadoop
0 hell
0 hadoop
0 hello
1 hadoop

Overwriting doc.txt


In [92]:
import os
notebook_directory = os.path.abspath(os.getcwd())
print("Notebook directory:", notebook_directory)

Notebook directory: /Users/neteasegames/Desktop/hadoop_mapreduce/doc_in_collection


In [93]:
!rm -rf {os.path.join(notebook_directory, 'output')}

In [94]:
# brew install hadoop
!hadoop jar /opt/homebrew/Cellar/hadoop/3.4.0/libexec/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-input file://{os.path.join(notebook_directory, 'doc.txt')} \
-output file://{os.path.join(notebook_directory, 'output')} \
-file {os.path.join(notebook_directory, 'mapper.py')} \
-file {os.path.join(notebook_directory, 'reducer.py')} \
-file {os.path.join(notebook_directory, 'combiner.py')} \
-mapper '/Users/neteasegames/anaconda3/bin/python mapper.py' \
-reducer '/Users/neteasegames/anaconda3/bin/python reducer.py' \
-combiner '/Users/neteasegames/anaconda3/bin/python combiner.py'

2024-05-01 11:27:01,456 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
2024-05-01 11:27:01,566 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/Users/neteasegames/Desktop/hadoop_mapreduce/doc_in_collection/mapper.py, /Users/neteasegames/Desktop/hadoop_mapreduce/doc_in_collection/reducer.py, /Users/neteasegames/Desktop/hadoop_mapreduce/doc_in_collection/combiner.py, /var/folders/j4/j3x8v2856p95gl278v7rl4j80000gn/T/hadoop-unjar7771402297968905089/] [] /var/folders/j4/j3x8v2856p95gl278v7rl4j80000gn/T/streamjob12112375089825970283.jar tmpDir=null
2024-05-01 11:27:02,069 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2024-05-01 11:27:02,187 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2024-05-01 11:27:02,350 INFO mapreduce.JobResourceUploader: Disabl

In [95]:
# 00000 is the reducer
!cat output/part-*

hadoop	1
hello	1


## Mapper 1 Combiner Input
hadoop	1

hadoop	0

hell	0

hello	0

## Mapper 2 Combiner Input
hadoop	1

hadoop	0

hadoop	0

hello	1

hellohello	0

# Reduce Input
hadoop	0

hadoop	1

hadoop	0

hadoop	1

hell	0

hello	0

hello	1

hellohello	0